In [2]:
using CairoMakie, Dierckx, Optim, LinearAlgebra, QuantEcon

In [3]:
pars = (;α = 0.33, # Capital share
        β = 0.9, # Discount factor
        A = 10.0, # TFP
        γ = 1.0, # Risk aversion
        δ = 0.1, # Depreciation rate
        nk = 31, # Number of capital gridpoints
        θ = 3, # Grid expansion parameter
        lb = 0.00, # Lower bound of capital grid
        ub = 1000.0, # Upper bound of capital grid
        nz = 19, # Number of shock gridpoints
        ρ = 0.9, # Persistence of AR(1) process
        μ = 0.0, # Mean of AR(1) process
        σ = 0.007, # Var of AR(1) process
        toler = 1e-6, # Tolerance
        maxiter = 10000) # Maximum number of iterations

(α = 0.33, β = 0.9, A = 10.0, γ = 1.0, δ = 0.1, nk = 31, θ = 3, lb = 0.0, ub = 1000.0, nz = 19, ρ = 0.9, μ = 0.0, σ = 0.007, toler = 1.0e-6, maxiter = 10000)

In [18]:
function utility(c)#, pars)
    #(; γ) = pars
    if pars.γ == 1.0
        return log(c)
    else
        return (c^(1-γ)) / (1-γ)
    end
end

function ar1(pars)
    (; μ, ρ, σ, nz) = pars
    mc = rouwenhorst(nz, μ, ρ, σ)
    Π, Zvals = mc.p, exp.(mc.state_values)
    return Π, Zvals
end

function exp_grid(pars)
    (; nk, θ, lb, ub) = pars
    grid = LinRange(1e-4, 1.0, nk)
    expgrid = lb .+ (ub .- lb) .* grid.^θ
    return expgrid
end

function interpolate(grid, vals, pars)
    spline = Spline1D(grid, vals, k = 3, bc = "extrapolate")
    return spline
end

function production(k, z ,pars)
    (; α, δ, A) = pars
    return A * z * (k ^ α) + (1.0 - δ) * k
end

function initial_guess(grid, pars)
    (; nk, nz) = pars
    v = ones(nk, nz)
    v_out = zeros(nk, nz)
    for j in 1:nz
        v_out[:,j] = utility.(grid)#, pars)
    end
    return v_out
end

#test_Π, test_Zvals = ar1(pars)
#test_grid = exp_grid(pars)
#test_v = ones(pars.nk)
#test_interp = interpolate(test_grid, test_v, pars)
#test_deriv = Dierckx.derivative(test_interp, 15.0)
#test_production = production(test_grid[19], test_Zvals[1], pars)
#test_init = initial_guess(test_grid, pars)

195.74246522684115

In [ ]:
function egm(pars)
    